In [1]:
import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rapids_singlecell as rsc
import scanpy as sc
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

/mnt/cephfs/home/users/krakovic/.conda/envs/scpy/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
def subsample_cores(frame, subsample):
    unique_cores_arr = np.unique(frame['core'])
    np.random.seed(1)
    random_indices = np.random.choice(unique_cores_arr.size, subsample, replace=False)
    subsampled_cores = np.unique(unique_cores_arr[random_indices])
    train_frame = frame[frame['core'].isin(subsampled_cores)]
    return train_frame

def sanity_check_neighbors(adata, dim_columns, neighbors_key='nn_leiden', tabs='\t\t'):
	distances = adata.obsp['%s_distances' % neighbors_key]
	n_neighbors = adata.uns['nn_leiden']['params']['n_neighbors']

	# Get locations with issues.
	original_frame_locs = list()
	i = 0
	for row in distances.tolil().rows:
		if len(row) != n_neighbors - 1:
			original_frame_locs.append(i)
		i += 1

	# If there's no problematic instances, continue
	if len(original_frame_locs) == 0:
		return False, None

	print('%sFound %s problematic instances' % (tabs, len(original_frame_locs)))
	print('%sRe-running clustering.' % tabs)
	# Recover from adata
	frame_sub = pd.DataFrame(adata.X, columns=dim_columns)
	for column in adata.obs.columns:
		frame_sub[column] = adata.obs[column].values

	# Drop problematic instances
	frame_sub = frame_sub.drop(original_frame_locs)
	return True, frame_sub

def plot_leiden_clusters_umap(resolutions, adata, csv_file):
    csv_dir = os.path.dirname(csv_file)
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
    # axs = axs.flatten()

    sc.tl.umap(adata, min_dist=0.5, spread=1.0, n_components=2, neighbors_key='nn_leiden', method='rapids')

    # for i, res in enumerate(resolutions):
    sc.pl.umap(adata, color=f'leiden_{resolution}', show=False, ax=axs)

    plt.tight_layout()
    plt.gcf()
    groupby = str(resolution).replace('.', 'p')
    plt.savefig(fname=os.path.join(csv_dir, f'01_training_{subsample}_cores_umap_leiden_{groupby}.png'))

def plot_leiden_clusters_boxplot(adata, resolution, markers, subsample, csv_file):
    csv_dir = os.path.dirname(csv_file)
    fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(40,40), sharey=True)
    axs = axs.flatten()

    for i, leiden in enumerate(sorted(adata.obs[f'leiden_{resolution}'].unique())):
        subset = adata.X[adata.obs[f'leiden_{resolution}'] == leiden]
        sns.boxplot(data=subset, width=0.5, ax=axs[i])
        axs[i].set_title(f'{leiden}')
        axs[i].tick_params(axis='x', labelrotation=45)
        axs[i].set_xticks(ticks=axs[i].get_xticks(), labels=markers)
        axs[i].axhline(y=0)
    
    plt.tight_layout()
    plt.gcf()
    groupby = str(resolution).replace('.', 'p')
    if subsample == True:
        plt.savefig(fname=os.path.join(csv_dir, f'02_training_{subsample}_cores_{groupby}_boxplot.png'))
    else:
        plt.savefig(fname=os.path.join(csv_dir, f'03_test_all_cores_leiden_{groupby}_boxplot.png'))

In [10]:
# This is the output from the previous notebook, with cell coordinates and normalised expression data

csv_file = '/mnt/cephfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/TMAs/assign_bioclavis_to_clusters_post_rotation/all_tma_data_filteredROIs_042024_ki67_dropna.csv'
subsample = None
save_h5ad = True

resolutions = [0.2, 0.3, 0.5]
markers = ['CKPAN_z', 'CD4_z', 'CD8_z', 'CD68_z', 'SMA_z', 'Ki67_z']

In [11]:
frame = pd.read_csv(csv_file, index_col=0)
frame

,TMA,core,compartment,CellX,CellY,CKPAN_z,CD4_z,CD8_z,CD68_z,SMA_z,Ki67_z
1,22,22-05-J,Tumour,8526.375294,-47209.294961,2.308457,-0.450230,-0.204453,-0.302017,-0.049015,-0.225220
2,22,22-05-J,Tumour,8502.780534,-47222.466136,3.451042,-0.263806,-0.194394,-0.387213,1.035879,-0.227051
3,22,22-05-J,Tumour,8510.929662,-47224.074935,2.488115,-0.475064,-0.279479,-0.315544,-0.155966,0.633958
4,22,22-05-J,Tumour,8517.623256,-47224.496313,2.098992,-0.376636,-0.268872,-0.287480,0.892927,-0.052510
5,22,22-05-J,Stroma,8522.685445,-47229.076912,0.711897,-0.517915,-0.206411,1.691172,0.079570,0.027650
...,...,...,...,...,...,...,...,...,...,...,...
776711,7,07-03-E,Tumour,16055.293306,-60032.408448,-0.546098,0.278704,2.150438,2.299076,-0.126520,-0.190065
776712,7,07-03-E,Stroma,16074.000349,-60032.173736,-0.490777,-0.108997,2.573555,-0.118711,-0.028570,-0.292293
776713,7,07-03-E,Stroma,16094.125869,-60035.216743,-0.399116,-0.183728,-0.164026,-0.202981,0.727282,-0.264193
776714,7,07-03-E,Tumour,16043.593802,-60034.513577,-0.440560,-0.336552,2.331294,-0.441513,0.132079,-0.279672


In [12]:
if subsample is not None:
    # Subsample entire cores for training
    print(f"Subsampling {subsample} cores")
    train_frame = subsample_cores(frame=frame, subsample=subsample)
else:
    train_frame = frame

adata_train = ad.AnnData(X=train_frame[markers], obs=train_frame[['TMA', 'core', 'compartment', 'CellX', 'CellY',]])

sc.pp.neighbors(adata_train, n_neighbors=15, method='rapids', metric='euclidean', key_added='nn_leiden')

# print(f"Generating full dataset AnnData matrix")
# adata = ad.AnnData(X=frame[markers], obs=frame[['TMA', 'core', 'compartment', 'CellX', 'CellY',]])

In [13]:
for resolution in resolutions:
    print(f"Training at Leiden {resolution}")
    rsc.tl.leiden(adata_train, resolution=resolution, key_added=f'leiden_{resolution}', neighbors_key='nn_leiden')
    # plot_leiden_clusters_boxplot(adata=adata_train, resolution=resolution, markers=markers, subsample=True, csv_file=csv_file)
    # plot_leiden_clusters_umap(resolutions=resolution, adata=adata_train, csv_file=csv_file)
    if subsample is not None:
        print(f"Processing test set at Leiden {resolution}")
        sc.tl.ingest(adata=adata, adata_ref=adata_train, obs=f'leiden_{resolution}', embedding_method='umap', neighbors_key='nn_leiden')
        plot_leiden_clusters_boxplot(adata=adata, resolution=resolution, markers=markers, subsample=False, csv_file=csv_file)

if save_h5ad == True:
    csv_dir = os.path.dirname(csv_file)
    adata_train.write_h5ad(os.path.join(csv_dir, f'hdf5_bioclavis_all_clustered_train_{subsample}_subsample_leiden_0p5.h5ad'))
    # adata.write_h5ad(os.path.join(csv_dir, 'hdf5_bioclavis_all_clustered_complete.h5ad'))

Training at Leiden 0.2
Training at Leiden 0.3
Training at Leiden 0.5
